In [5]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pdfplumber
import nltk
import numpy as np
import nltk
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer 
import heapq

In [8]:
pdf_txt = """Life insurance is a legally enforceable contract between two parties both of whom are legally qualified to contract. It is therefore, necessary that the terms and conditions of the agreement must be suitably documented in a manner that would make it clear that both parties to the contract are Ad- idem i.e., of the same mind. Ad-Idem means that both the parties understand the same thing in the same sense or are of the same mind on the same subject. There must be consensus or Ad-Idem between the parties to the contract.
This is possible provided all the terms and conditions, rights and duties - privileges and obligations are properly documented in terms which can be clearly interpreted in a court of law. Between two human beings sometime silence means an acceptance. But as the insurer is a legal personality entitled to contract verbal discussion between parties to the contract is not possible and hence there is a need for documentation.
Insurance is also a contract of utmost good faith and enforced only in the distant future. It is therefore necessary that the declarations made by both the parties should be put in black and white for future reference. Any suppression, willful and material shall make the contract void. The insured, therefore, has a duty to declare all that he knows about himself, his health, his financial status in answering questions contained in the proposal form and other ancillary documents which may be required by the insurer.
Age is an important factor in deciding the quantum of premium against a policy. The document proving the age, i.e. age proof must be reliable and the insured has to undertake as to its truthfulness.
Non-standard age proofs are those which are comparatively less reliable and therefore the insurer accepts them with a pinch of salt. In other words the insurer takes certain precautions before accepting such age proofs as final.
Proof of income is the document may become necessary whenever the sum proposed is very high. Normally a sum proposed which is seven to eight times of the declared income is acceptable for insurance. But proposals do come to the insurer when the known source of income of the proposer is much less compared to the amount of insurance desired. A service holder normally does not face this problem as his sources of income are verifiable.
In case of business people, the assessed income is at times much less compared to what is a desirable income for the amount of insurance desired. In such cases the insurer at times calls for assessed income tax returns, or Chartered Accountant’s certificate etc. Such precautions are necessary to eliminate the possibility of moral hazard.
Policy Contract is the policy document is a detailed document and it is the Evidence of the insurance contract which mentions all the terms and conditions of the insurance. The insured buys not the policy contract, but the right to the sum of money and its future delivery. The insurer on its part promises to pay a sum of money, provided of course the insured keeps its part of promise of paying the installments of premium as scheduled.
The pre-amble to the insurance contract makes the above statement clear and states that this policy is issued subject to the conditions and privileges printed on the back of the policy. The endorsements placed on the policy shall also be part of the policy and it also makes a reference to the proposal form saying that that the statements given in the proposal form are the basis of the contract.
The schedule which is printed on the policy document identifies the office which has issued the policy. It states the name of the policyholder, the date of commencement of the policy, an identification number of the policy called policy number. This number is extremely useful for making any reference to the insurer relating to this policy. This shall avoid needless delay.
Beneficiary’s name is also mentioned along with address.
It is necessary to check that it is correct and any mistake should be immediately pointed out for correction. A mistake in the address may misdirect the premium notices and any other future correspondence. It also states the name of the nominee and the date upto which premium has to be paid. The schedule goes on to mention, the type of policy, on the happening of which, the sum assured is payable and to whom it is payable. It of course also mentions when and how long the premium is to be paid.
The policy document is signed by an official of the insurer and dated and stamped as per the provision of the Stamp Act to make it a completely legally enforceable document.
An assignment of a policy in favour of another person or institution can be effected by an endorsement on the policy. Re-assignment can also be done by a subsequent endorsement on the same policy.As a nomination is automatically cancelled due to an assignment, after re-assignment, it is necessary to make a fresh nomination.
In Duplicate policy a policy document is a valuable document and can be used for mortgage etc. Loss of policy document does not absolve the insurer from the liability of payment of policy proceeds when the claim arises. The claim can be settled on the claimants, furnishing an indemnity bond jointly with one surety.
If a policy is irrevocably lost, a duplicate policy can be issued, after following a certain procedure like the insurer satisfies itself of the circumstances leading to loss and being so satisfied the insurer insists upon an advertisement in a news paper, production of an indemnity bond and payment of policy preparation charges and there after a duplicate policy is issued.
Generally nomination is made at the time of taking a policy. In case it is not done, it is possible to make nomination subsequently by an endorsement on the policy. It is also possible to change a nomination subsequently by an endorsement. After marriage, such change in nomination is normally required.
Life insurance being a legally enforceable contract, needs to be documented with details of the rights and obligations of the parties to the contract. Proposal form duly filled in and signed by the proposer is the first document which forms the basis of the contract.
Every time, the insured pays the premium, he receives a premium receipt. The premium needs to be paid in time, non- payment of premium leads to policy-lapses. Re-instatement of the cover is called revival of the policy.
If the policy is not revived, the policy can become a paid up policy for a reduced sum assured under certain conditions.
The policy document mentions in detail all the rights and obligations of the policyholder. The agent is advised to explain the various provisions of the policy to the policyholder.
The wordings in the policy document are of technical nature and hence the need for explaining. If there are certain endorsements on the policy, that need to be explained too.
It needs to be explained that the policy is a valuable document and needs to be kept in safe custody and in the knowledge of the close relatives.
"""

import nltk
nltk.download('punkt')
tokens = nltk.sent_tokenize(pdf_txt)
for t in tokens:
    print(t, "\n")
     

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Life insurance is a legally enforceable contract between two parties both of whom are legally qualified to contract. 

It is therefore, necessary that the terms and conditions of the agreement must be suitably documented in a manner that would make it clear that both parties to the contract are Ad- idem i.e., of the same mind. 

Ad-Idem means that both the parties understand the same thing in the same sense or are of the same mind on the same subject. 

There must be consensus or Ad-Idem between the parties to the contract. 

This is possible provided all the terms and conditions, rights and duties - privileges and obligations are properly documented in terms which can be clearly interpreted in a court of law. 

Between two human beings sometime silence means an acceptance. 

But as the insurer is a legal personality entitled to contract verbal discussion between parties 

In [9]:
def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences

In [10]:

cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
print(cleaned_sentences)
print(cleaned_sentences_with_stopwords)
     

['life insurance legally enforceable contract parties legally qualified contract', 'necessary terms conditions agreement suitably documented manner clear parties contract ad idem mind', 'adidem means parties understand thing sense mind subject', 'consensus adidem parties contract', 'possible provided terms conditions rights duties privileges obligations properly documented terms clearly interpreted court law', 'human beings silence means acceptance', 'insurer legal personality entitled contract verbal discussion parties contract possible need documentation', 'insurance contract utmost good faith enforced distant future', 'necessary declarations parties black white future reference', 'suppression willful material shall contract void', 'insured duty declare knows health financial status answering questions contained proposal form ancillary documents required insurer', 'age important factor deciding quantum premium policy', 'document proving age', 'age proof reliable insured undertake tru

> **A corpus is an extensive and well-structured collection of texts that a machine can read**

In [11]:
import numpy
sentences = cleaned_sentences_with_stopwords
sentence_words = [[word for word in document.split()]
                  for document in sentences]

from gensim import corpora
dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
  print(key, ' : ', value)

0  :  a
1  :  are
2  :  between
3  :  both
4  :  contract
5  :  enforceable
6  :  insurance
7  :  is
8  :  legally
9  :  life
10  :  of
11  :  parties
12  :  qualified
13  :  to
14  :  two
15  :  whom
16  :  ad
17  :  agreement
18  :  and
19  :  be
20  :  clear
21  :  conditions
22  :  documented
23  :  idem
24  :  ie
25  :  in
26  :  it
27  :  make
28  :  manner
29  :  mind
30  :  must
31  :  necessary
32  :  same
33  :  suitably
34  :  terms
35  :  that
36  :  the
37  :  therefore
38  :  would
39  :  adidem
40  :  means
41  :  on
42  :  or
43  :  sense
44  :  subject
45  :  thing
46  :  understand
47  :  consensus
48  :  there
49  :  all
50  :  can
51  :  clearly
52  :  court
53  :  duties
54  :  interpreted
55  :  law
56  :  obligations
57  :  possible
58  :  privileges
59  :  properly
60  :  provided
61  :  rights
62  :  this
63  :  which
64  :  acceptance
65  :  an
66  :  beings
67  :  human
68  :  silence
69  :  sometime
70  :  as
71  :  but
72  :  discussion
73  :  documentation

## Bag of Words (BOW)

The first model we will use for semantic similarity is leveraging Bag of Words (BOW). With BOW, each sentence is encoded into a vector whose length is the number of words in the vocabulary. Each element of the vector indicates how many times the particular word occurs in the sentence. An example is shown below by printing the dictionary and the FAQ questions in the BOW sparse format.

**Note that a vector representation of a sentence is often also called an "Embedding" since it is a way of embedding a sentence in M-dimensional space if the vector is of length M.**

In [12]:

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent, embedding in zip(sentences, bow_corpus):
  print(sent)
  print(embedding)

life insurance is a legally enforceable contract between two parties both of whom are legally qualified to contract
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
it is therefore necessary that the terms and conditions of the agreement must be suitably documented in a manner that would make it clear that both parties to the contract are ad idem ie of the same mind
[(0, 1), (1, 1), (3, 1), (4, 1), (7, 1), (10, 2), (11, 1), (13, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 4), (37, 1), (38, 1)]
adidem means that both the parties understand the same thing in the same sense or are of the same mind on the same subject
[(1, 1), (3, 1), (10, 1), (11, 1), (25, 1), (29, 1), (32, 4), (35, 1), (36, 5), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46,

In [13]:

user_question = "When can a duplicate policy be issued?"

In [14]:

question = clean_sentence(user_question, stopwords=False)
question_embedding = dictionary.doc2bow(question.split())
print(user_question, "\n", question_embedding)

When can a duplicate policy be issued? 
 [(0, 1), (19, 1), (50, 1), (132, 1), (183, 1), (228, 1), (308, 1)]


### Once we find a vector representation for each sentence using BOW,we can compute the distance between two vectors by taking the cosine similarity. Note other similarity measure can be used as well, but we will stick to cosine similarity throughout for simplicity.

The closest matching answer can be retrieved by finding the cosine similarity of the query vector with each of the FAQ question vectors.

In [15]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
  max_sim = -1
  index_sim = -1
  for index, embedding in enumerate(sentence_embeddings):
    sim = cosine_similarity(embedding, question_embedding)[0][0]
    print(index, sim, sentences[index])
    if sim > max_sim:
      max_sim = sim
      index_sim = index
  
  return index_sim
index = retrieveAndPrintFAQAnswer(question_embedding, bow_corpus, sentences)

0 1.0 life insurance is a legally enforceable contract between two parties both of whom are legally qualified to contract
1 1.0 it is therefore necessary that the terms and conditions of the agreement must be suitably documented in a manner that would make it clear that both parties to the contract are ad idem ie of the same mind
2 0.7071067811865475 adidem means that both the parties understand the same thing in the same sense or are of the same mind on the same subject
3 0.4472135954999579 there must be consensus or adidem between the parties to the contract
4 1.0 this is possible provided all the terms and conditions rights and duties  privileges and obligations are properly documented in terms which can be clearly interpreted in a court of law
5 0.4472135954999579 between two human beings sometime silence means an acceptance
6 1.0 but as the insurer is a legal personality entitled to contract verbal discussion between parties to the contract is not possible and hence there is a nee

In [73]:
print("Question: ", question)
print("Answer: ", sentences[index])

Question:  How to improve the quality of knowledge extraction?
Answer:  it needs to be explained that the policy is a valuable document and needs to be kept in safe custody and in the knowledge of the close relatives


### BOW representation did not do very well and retrieved the wrong answer since it is looking for exact word match.

# Using Word2Vec Embedding

In [17]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = None
try:
  v2w_model = gensim.models.Keyedvectors.load('./w2vecmodel.mod')
  print("w2v Model Successfully loaded")
except:
  v2w_model = api.load('word2vec-google-news-300')
  v2w_model.save("./w2vecmodel.mod")
  print("w2v Model Saved")

w2vec_embedding_size = len(v2w_model['pc']) 

[=============================================-----] 91.9% 1528.2/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.4% 1653.3/1662.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
def getWordVec(word, model):
  samp = model['pc']
  vec = [0]*len(samp)
  try:
    vec = model[word]
  except:
    vec = [0]*len(samp)
  return (vec)


def getPhraseEmbedding(phrase, embeddingmodel):
  samp = getWordVec('computer', embeddingmodel)
  vec = numpy.array([0]*len(samp))
  den = 0;
  for word in phrase.split():
    den = den+1
    vec = vec+numpy.array(getWordVec(word, embeddingmodel))
  return vec.reshape(1, -1)

In [19]:
sent_embeddings = []
for sent in sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, v2w_model))

question_embedding = getPhraseEmbedding(question, v2w_model)
index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)

0 0.5513437234038394 life insurance is a legally enforceable contract between two parties both of whom are legally qualified to contract
1 0.7025787291556693 it is therefore necessary that the terms and conditions of the agreement must be suitably documented in a manner that would make it clear that both parties to the contract are ad idem ie of the same mind
2 0.6070676295451412 adidem means that both the parties understand the same thing in the same sense or are of the same mind on the same subject
3 0.6058007155185451 there must be consensus or adidem between the parties to the contract
4 0.6729521680764039 this is possible provided all the terms and conditions rights and duties  privileges and obligations are properly documented in terms which can be clearly interpreted in a court of law
5 0.430919800661395 between two human beings sometime silence means an acceptance
6 0.6073519392489676 but as the insurer is a legal personality entitled to contract verbal discussion between parti

In [20]:
print("Question: ", question)
print("Answer: ", cleaned_sentences_with_stopwords[index])

Question:  when can a duplicate policy be issued
Answer:  if a policy is irrevocably lost a duplicate policy can be issued after following a certain procedure like the insurer satisfies itself of the circumstances leading to loss and being so satisfied the insurer insists upon an advertisement in a news paper production of an indemnity bond and payment of policy preparation charges and there after a duplicate policy is issued


# Glove Embedding

### GloVe (Global Vectors for Word Representation) is a word embedding technique used in natural language processing to represent words as dense vectors in a high-dimensional space.

GloVe differs from other word embedding methods, like Word2Vec, in how it constructs word vectors. Instead of solely relying on local context (neighboring words), GloVe leverages global statistical information from a large corpus of text.

Here's a simplified explanation of how GloVe works:

* GloVe starts by creating a co-occurrence matrix that captures how frequently words co-occur with each other in the given text corpus.
* It then applies matrix factorization techniques to this co-occurrence matrix. The goal is to factorize the matrix into two smaller matrices, such that their product approximates the original co-occurrence matrix.
* The resulting factorized matrices are used to compute word vectors. Each word is represented by a vector in the high-dimensional space, where the dimensions capture the relationships between words based on their co-occurrence patterns.
* By training on a large corpus, GloVe learns to capture both the global context (how words relate to each other across the entire corpus) and the local context (how words relate to their neighboring words).
* The learned word vectors can be used in various natural language processing tasks, such as text classification, machine translation, and sentiment analysis.

### GloVe embeddings have several advantages. They tend to capture semantic relationships between words, such as analogies (e.g., "king" - "man" + "woman" ≈ "queen"). Additionally, they are less sensitive to word frequency and work well even with rare words.

Overall, GloVe provides a powerful way to represent words as vectors by leveraging global statistical information from a corpus, enabling computers to understand and work with language more effectively.




In [21]:
import gensim.downloader as api

glove_model = None
try:
  glove_model = gensim.models.Keyedvectors.load('./glovemodel.mod')
  print("Glove Model Successfully loaded")
except:
  glove_model = api.load('glove-twitter-25')
  glove_model.save("./glovemodel.mod")
  print("Glove Model Saved")

glove_embedding_size = len(glove_model['pc'])

Glove Model Saved


In [22]:
def getWordVec(word, model):
  samp = model['pc']
  vec = [0]*len(samp)
  try:
    vec = model[word]
  except:
    vec = [0]*len(samp)
  return (vec)


def getPhraseEmbedding(phrase, embeddingmodel):
  samp = getWordVec('computer', embeddingmodel)
  vec = numpy.array([0]*len(samp))
  den = 0;
  for word in phrase.split():
    den = den+1
    vec = vec+numpy.array(getWordVec(word, embeddingmodel))
  return vec.reshape(1, -1)

In [23]:
sent_embeddings = []
for sent in cleaned_sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, glove_model))

question_embedding = getPhraseEmbedding(question, glove_model)
index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)

0 0.9022455392443607 life insurance is a legally enforceable contract between two parties both of whom are legally qualified to contract
1 0.9084665468647729 it is therefore necessary that the terms and conditions of the agreement must be suitably documented in a manner that would make it clear that both parties to the contract are ad idem ie of the same mind
2 0.9324847200286663 adidem means that both the parties understand the same thing in the same sense or are of the same mind on the same subject
3 0.8267337040436453 there must be consensus or adidem between the parties to the contract
4 0.8828094588463636 this is possible provided all the terms and conditions rights and duties  privileges and obligations are properly documented in terms which can be clearly interpreted in a court of law
5 0.849249967948168 between two human beings sometime silence means an acceptance
6 0.9104672345379422 but as the insurer is a legal personality entitled to contract verbal discussion between parti

In [24]:
print("Question: ", question)
print("Answer: ", cleaned_sentences_with_stopwords[index])

Question:  when can a duplicate policy be issued
Answer:  it needs to be explained that the policy is a valuable document and needs to be kept in safe custody and in the knowledge of the close relatives


# Fine tuned Bert Technique

Bidirectional Encoder Representations from Transformers

Instead of looking at words in isolation, BERT, a transformer based model attempts to use the context of words to get embeddings. BERT uses several concepts of deep learning come up with a model that looks at context in a bidirectional fashion, leveraging information from the entire sentences as a whole through self-attention.

Example: Suppose Question is "How I can cross a road when their is no zebra crossing." and this question very similar to query "How I can cross road on zebra crossing." and thier is a chance that above two method give wrong answers but could handle that question. And we don't remove stoping words in the document because if it remove word like in above example "no" meaning of whole sentance become change. Bert uses stoping words to make relations between words.

What is Model Fine-Tuning?

BERT (Bidirectional Encoder Representations from Transformers) is a big neural network architecture, with a huge number of parameters, that can range from 100 million to over 300 million. So, training a BERT model from scratch on a small dataset would result in overfitting.

So, it is better to use a pre-trained BERT model that was trained on a huge dataset, as a starting point. We can then further train the model on our relatively smaller dataset and this process is known as model fine-tuning.

Installing the Hugging Face library

In [25]:

!pip install transformers==3.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.0/884.0 kB 13.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      /tmp/pip-build-env-mtn8d6qo/overlay/lib/python3.10/site-packages/setuptools/dist.py:509: InformationOnly: Normalizing '0.8.1.rc2' to '0.8.1rc2'
        self.metadata.version = self._normalize_version(
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
   

In [26]:
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [27]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [38]:
answer_text = """The ability of the machine to infer knowledge from
the user documents can be tested based on its ability to answer
the question asked. Conventional Artificial Neural Network
(ANN) models for knowledge extraction only answer to the
questions which are simple and objective as they don't analyze
the questions and don't try to understand what really the content
of document mean. The proposed question answering system
(QAS) uses deep cases along with ANN to understand the
contents of the documents. We divide the sentences of natural
language into knowledge units and assign deep case to each word
to improve the quality of knowledge extraction."""

question = "How to improve the quality of knowledge extraction?"

In [39]:
input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
print('The input has a total of {:} tokens.'.format(len(input_ids)))


The input has a total of 130 tokens.


In [40]:
input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)
print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 130 tokens.


In [41]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    if id == tokenizer.sep_token_id:
        print('')
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')b
     

[CLS]           101
how           2,129
to            2,000
improve       5,335
the           1,996
quality       3,737
of            1,997
knowledge     3,716
extraction   14,676
?             1,029

[SEP]           102

the           1,996
ability       3,754
of            1,997
the           1,996
machine       3,698
to            2,000
in            1,999
##fer         7,512
knowledge     3,716
from          2,013
the           1,996
user          5,310
documents     5,491
can           2,064
be            2,022
tested        7,718
based         2,241
on            2,006
its           2,049
ability       3,754
to            2,000
answer        3,437
the           1,996
question      3,160
asked         2,356
.             1,012
conventional  7,511
artificial    7,976
neural       15,756
network       2,897
(             1,006
ann           5,754
)             1,007
models        4,275
for           2,005
knowledge     3,716
extraction   14,676
only          2,069
answer        3,43

We can concatenated the question and answer_text together, but BERT still needs a a way to distinguish them. BERT has two special "Segment" embeddings, one for segment "A" and one for segment "B". Before the word embeddings go into the BERT layers, the segment A embedding needs to be added to the question tokens, and the segment B embedding needs to be added to each of the answer_text tokens. These handled by the transformer library and all we need to do is specify '0' and '1' for the token.

In [52]:
sep_index = input_ids.index(tokenizer.sep_token_id)
num_seg_a = sep_index + 1
num_seg_b = len(input_ids) - num_seg_a
#Here We Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b
# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [55]:
input_ids

[101,
 2129,
 2000,
 5335,
 1996,
 3737,
 1997,
 3716,
 14676,
 1029,
 102,
 1996,
 3754,
 1997,
 1996,
 3698,
 2000,
 1999,
 7512,
 3716,
 2013,
 1996,
 5310,
 5491,
 2064,
 2022,
 7718,
 2241,
 2006,
 2049,
 3754,
 2000,
 3437,
 1996,
 3160,
 2356,
 1012,
 7511,
 7976,
 15756,
 2897,
 1006,
 5754,
 1007,
 4275,
 2005,
 3716,
 14676,
 2069,
 3437,
 2000,
 1996,
 3980,
 2029,
 2024,
 3722,
 1998,
 7863,
 2004,
 2027,
 2123,
 1005,
 1056,
 17908,
 1996,
 3980,
 1998,
 2123,
 1005,
 1056,
 3046,
 2000,
 3305,
 2054,
 2428,
 1996,
 4180,
 1997,
 6254,
 2812,
 1012,
 1996,
 3818,
 3160,
 10739,
 2291,
 1006,
 1053,
 3022,
 1007,
 3594,
 2784,
 3572,
 2247,
 2007,
 5754,
 2000,
 3305,
 1996,
 8417,
 1997,
 1996,
 5491,
 1012,
 2057,
 11443,
 1996,
 11746,
 1997,
 3019,
 2653,
 2046,
 3716,
 3197,
 1998,
 23911,
 2784,
 2553,
 2000,
 2169,
 2773,
 2000,
 5335,
 1996,
 3737,
 1997,
 3716,
 14676,
 1012,
 102]

In [57]:
outputs = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

In [54]:
end_scores

'end_logits'

In [58]:
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)

answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "divide the sentences of natural language into knowledge units and assign deep case to each word"


In [61]:
def answer_question(question, answer_text):

    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)

    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    sep_index = input_ids.index(tokenizer.sep_token_id)

    num_seg_a = sep_index + 1

    num_seg_b = len(input_ids) - num_seg_a

    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    assert len(segment_ids) == len(input_ids)

    outputs = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    start_scores, end_scores = outputs.start_logits, outputs.end_logits

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    #print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))
    #print(f'score: {torch.max(start_scores)}')
    score = float(torch.max(start_scores))
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]

    for i in range(answer_start + 1, answer_end + 1):

        if tokens[i][0:2] == ' ':
            answer += tokens[i][2:]

        else:
            answer += ' ' + tokens[i]
    return answer, score
    #print('Answer: "' + answer + '"')

In [62]:
answer = tokens[answer_start]
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
  # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
        # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]
print('Answer: "' + answer + '"')

Answer: "divide the sentences of natural language into knowledge units and assign deep case to each word"


In [63]:
answer_question(question, answer_text)

Query has 130 tokens.



('divide the sentences of natural language into knowledge units and assign deep case to each word',
 6.490675926208496)

In [28]:

tokens = tokenizer.encode_plus(user_question, pdf_txt, add_special_tokens=True, max_length=512, truncation=True, padding="max_length")
tokens

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


{'input_ids': [101, 2043, 2064, 1037, 24473, 3343, 2022, 3843, 1029, 102, 2166, 5427, 2003, 1037, 10142, 16306, 3085, 3206, 2090, 2048, 4243, 2119, 1997, 3183, 2024, 10142, 4591, 2000, 3206, 1012, 2009, 2003, 3568, 1010, 4072, 2008, 1996, 3408, 1998, 3785, 1997, 1996, 3820, 2442, 2022, 4848, 8231, 8832, 1999, 1037, 5450, 2008, 2052, 2191, 2009, 3154, 2008, 2119, 4243, 2000, 1996, 3206, 2024, 4748, 1011, 8909, 6633, 1045, 1012, 1041, 1012, 1010, 1997, 1996, 2168, 2568, 1012, 4748, 1011, 8909, 6633, 2965, 2008, 2119, 1996, 4243, 3305, 1996, 2168, 2518, 1999, 1996, 2168, 3168, 2030, 2024, 1997, 1996, 2168, 2568, 2006, 1996, 2168, 3395, 1012, 2045, 2442, 2022, 10465, 2030, 4748, 1011, 8909, 6633, 2090, 1996, 4243, 2000, 1996, 3206, 1012, 2023, 2003, 2825, 3024, 2035, 1996, 3408, 1998, 3785, 1010, 2916, 1998, 5704, 1011, 14310, 1998, 14422, 2024, 7919, 8832, 1999, 3408, 2029, 2064, 2022, 4415, 10009, 1999, 1037, 2457, 1997, 2375, 1012, 2090, 2048, 2529, 9552, 8811, 4223, 2965, 2019, 9920, 1

It return a dictionary containing three key-value pairs, input_ids , token_type_ids , and attention_mask . We have also added return_tensors='pt' to return PyTorch tensors from the tokenizer (rather than Python lists).

In [29]:

tokens = tokenizer.encode_plus(pdf_txt, add_special_tokens=False, return_tensors='pt')
tokens
     

Token indices sequence length is longer than the specified maximum sequence length for this model (1395 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([[2166, 5427, 2003,  ..., 2485, 9064, 1012]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [30]:

input_id_chunks = tokens['input_ids'][0].split(510)
mask_chunks = tokens['attention_mask'][0].split(510)

for tensor in input_id_chunks:
  print(len(tensor))

510
510
375


In [31]:
a = torch.arange(10)
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [32]:

torch.cat(
    [torch.Tensor([101]), a, torch.Tensor([102])]
)

tensor([101.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 102.])

In [33]:
chunksize = 512

input_id_chunks = list(tokens['input_ids'][0].split(chunksize - 2))
mask_chunks = list(tokens['attention_mask'][0].split(chunksize - 2))

for i in range(len(input_id_chunks)):
    input_id_chunks[i] = torch.cat([
        torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
    ])
    mask_chunks[i] = torch.cat([
        torch.tensor([1]), mask_chunks[i], torch.tensor([1])
    ])

    pad_len = chunksize - input_id_chunks[i].shape[0]
    if pad_len > 0:
        input_id_chunks[i] = torch.cat([
            input_id_chunks[i], torch.Tensor([0] * pad_len)
        ])
        mask_chunks[i] = torch.cat([
            mask_chunks[i], torch.Tensor([0] * pad_len)
        ])

for chunk in input_id_chunks:
    print(chunk)

tensor([  101,  2166,  5427,  2003,  1037, 10142, 16306,  3085,  3206,  2090,
         2048,  4243,  2119,  1997,  3183,  2024, 10142,  4591,  2000,  3206,
         1012,  2009,  2003,  3568,  1010,  4072,  2008,  1996,  3408,  1998,
         3785,  1997,  1996,  3820,  2442,  2022,  4848,  8231,  8832,  1999,
         1037,  5450,  2008,  2052,  2191,  2009,  3154,  2008,  2119,  4243,
         2000,  1996,  3206,  2024,  4748,  1011,  8909,  6633,  1045,  1012,
         1041,  1012,  1010,  1997,  1996,  2168,  2568,  1012,  4748,  1011,
         8909,  6633,  2965,  2008,  2119,  1996,  4243,  3305,  1996,  2168,
         2518,  1999,  1996,  2168,  3168,  2030,  2024,  1997,  1996,  2168,
         2568,  2006,  1996,  2168,  3395,  1012,  2045,  2442,  2022, 10465,
         2030,  4748,  1011,  8909,  6633,  2090,  1996,  4243,  2000,  1996,
         3206,  1012,  2023,  2003,  2825,  3024,  2035,  1996,  3408,  1998,
         3785,  1010,  2916,  1998,  5704,  1011, 14310,  1998, 

In [64]:
for tensor in range(len(input_id_chunks)):
  ans = answer_question(user_question, ' '.join(tokenizer.convert_ids_to_tokens(input_id_chunks[tensor])))
  print(ans)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


('[SEP]', 1.939035177230835)
Query has 512 tokens.



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


('[CLS] when can a duplicate policy be issued ? [SEP] [CLS] precautions are necessary to eliminate the possibility of moral hazard . policy contract is the policy document is a detailed document and it is the evidence of the insurance contract which mentions all the terms and conditions of the insurance . the ins # # ur ##ed buys not the policy contract , but the right to the sum of money and its future delivery . the ins # # ur ##er on its part promises to pay a sum of money , provided of course the ins # # ur ##ed keeps its part of promise of paying the installment # # s of premium as scheduled . the pre - am # # b ##le to the insurance contract makes the above statement clear and states that this policy is issued subject to the conditions and privileges printed on the back of the policy . the endorsement # # s placed on the policy shall also be part of the policy and it also makes a reference to the proposal form saying that that the statements given in the proposal form are the bas

“Let us start pretraining the model”

Now if we tokenize the sentence using wordpiece , then shall obtain

|token = [let , us , start , pre , ###train , ###ing , the , model]

while tokenizing the sentence , our word pretraining is splint into 3 parts , this happened because our word piece tokenizer first check whether the word is present in our vocabulary . If the word is present then it will used as a token but if not then our word is split into subwords recursively until the subwords are found in our corpus. This process is effective in handling the out of vocabulary words.

In [1]:
corrected_answer = ''

for word in ans.split():
    
    #If it's a subword token
    if word[0:2] == '##':
        corrected_answer += word[2:]
    else:
        corrected_answer += ' ' + word

print(corrected_answer)

NameError: name 'ans' is not defined

In [70]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2g74koiv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-2g74koiv
  Resolved https://github.com/huggingface/transformers.git to commit b52a03cd3bec92d0ee84f0b1f7edee0d5117200a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7262108 sha256=6880e0eea5974e4063853a3b08059abf4a13d040813ca2aa710876dcda625357
  Stored in directory: /tmp/pip-ephem-wheel-cache-nof0aw5j/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninstalling transformers-4.30.1:
      Successfully uninstalled transformers-4.30.1
  Cloning https://github.com